In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler # <-- ESSENCIAL PARA REDES NEURAIS
from sklearn.metrics import (
    accuracy_score, 
    classification_report, 
    confusion_matrix
)

# Configurar para reprodutibilidade
np.random.seed(42)
tf.random.set_seed(42)

# ==============================================================================
# --- 1. Carregando e Pré-processando os Dados ---
# ==============================================================================
print("--- 1. Carregando e Pré-processando os Dados ---")

# ... (Seu código de carregamento e pré-processamento) ...
# (Ocultado por brevidade, mas é o mesmo código que você já tem)
# ... (Seu código de carregamento e pré-processamento) ...

# Carregar o dataset
file_path = '../data/raw/leish_dataset.csv'
df = pd.read_csv(file_path)

# Criar cópia para processamento
df_processed = df.copy()

# Lidar com valores ausentes (Missing)
for col in df_processed.select_dtypes(include=['object']).columns:
    df_processed[col] = df_processed[col].fillna('Unknown')

# Codificar a variável Alvo (Target)
target_map = {'positivo': 1, 'negativo': 0, 'Unknown': 0}
df_processed['diagnosis'] = df_processed['diagnosis'].map(target_map).astype(int)

# Separar features (X) e alvo (y)
X_categorical = df_processed.drop('diagnosis', axis=1)
y = df_processed['diagnosis']

# Aplicar One-Hot Encoding nas features categóricas
X_numeric = pd.get_dummies(X_categorical, drop_first=True, dtype=int)
n_features = X_numeric.shape[1] # Salvar o número de features para a rede

print(f"--- Pré-processamento Concluído. {n_features} features criadas. ---")


# ==============================================================================
# --- 2. Dividindo os Dados (Treino e Teste) ---
# ==============================================================================
print("\n--- 2. Dividindo os Dados (Treino e Teste) ---")

X_train, X_test, y_train, y_test = train_test_split(
    X_numeric, 
    y, 
    test_size=0.3,
    random_state=42,
    stratify=y
)

print(f"Tamanho do conjunto de Treino: {X_train.shape[0]}")
print(f"Tamanho do conjunto de Teste:  {X_test.shape[0]}")


# ==============================================================================
# --- 3. [NOVO] Escalabilidade dos Dados (Scaling) ---
# ==============================================================================
print("\n--- 3. Aplicando StandardScaler (Fundamental para NNs) ---")

# Redes Neurais são muito sensíveis à escala dos dados (ex: uma feature "idade"
# de 0-80 não pode competir com uma feature "sexo" de 0-1).
# Precisamos padronizar (média=0, desvio=1).

scaler = StandardScaler()

# ATENÇÃO: Ajustamos (fit) o scaler APENAS nos dados de TREINO
X_train_scaled = scaler.fit_transform(X_train)

# E APENAS transformamos (transform) os dados de TESTE com o mesmo scaler
X_test_scaled = scaler.transform(X_test)

print("--- Dados escalados com sucesso ---")


# ==============================================================================
# --- 4. [NOVO] Calculando Pesos da Classe (class_weight) ---
# ==============================================================================
print("\n--- 4. Calculando Pesos da Classe (class_weight) ---")

# O Keras usa os pesos de forma diferente do scikit-learn.
# Vamos calculá-los manualmente para o `model.fit()`
unique, counts = np.unique(y_train, return_counts=True)
count_neg = counts[0] # 224
count_pos = counts[1] # 95
total = count_neg + count_pos # 319

# Fórmula para calcular pesos (Manual)
weight_for_0 = (1 / count_neg) * (total / 2.0)
weight_for_1 = (1 / count_pos) * (total / 2.0)

# Criar o dicionário de pesos
class_weight_dict = {0: weight_for_0, 1: weight_for_1}

print(f"Total de Treino: {total}, Negativos: {count_neg}, Positivos: {count_pos}")
print(f"Peso para classe 0 (negativo): {weight_for_0:.4f}")
print(f"Peso para classe 1 (positivo): {weight_for_1:.4f}")


# ==============================================================================
# --- 5. Modelo 7: Construindo a Rede Neural (Keras) ---
# ==============================================================================
print("\n--- 5. Modelo 7: Construindo a Arquitetura da Rede Neural ---")

model = Sequential()

# Camada de Entrada (Input) e Primeira Camada Oculta (Hidden)
# 'input_shape' deve ser o número de features
# 'relu' é a função de ativação padrão
model.add(Dense(32, activation='relu', input_shape=(n_features,)))
# Camada de Dropout para combater overfitting
model.add(Dropout(0.5)) # "Desliga" 50% dos neurônios nesta camada

# Segunda Camada Oculta
model.add(Dense(16, activation='relu'))
model.add(Dropout(0.5)) # Mais dropout

# Camada de Saída (Output)
# 'sigmoid' é usado para classificação binária (retorna um valor entre 0 e 1)
model.add(Dense(1, activation='sigmoid'))

# Compilar o modelo
# 'adam' é um otimizador robusto
# 'binary_crossentropy' é a função de perda correta para 'sigmoid'
model.compile(
    optimizer='adam', 
    loss='binary_crossentropy', 
    metrics=['accuracy', tf.keras.metrics.Recall(name='recall')] # Vamos monitorar o Recall!
)

# Mostrar um resumo da arquitetura
model.summary()


# ==============================================================================
# --- 6. Treinando o Modelo 7 (Rede Neural) ---
# ==============================================================================
print("\n--- 6. Treinando o Modelo 7 (Rede Neural) ---")

# Callback de EarlyStopping:
# Monitora 'val_loss' (perda nos dados de validação)
# 'patience=15': Para o treino se a 'val_loss' não melhorar por 15 épocas seguidas
# 'restore_best_weights=True': Salva o modelo do melhor ponto (não o último)
early_stopper = EarlyStopping(
    monitor='val_loss', 
    patience=15, 
    verbose=1, 
    restore_best_weights=True
)

# Treinar o modelo
# 'validation_split=0.2': Separa 20% dos dados de TREINO para validar o overfitting
# 'class_weight': Nosso dicionário de pesos!
history = model.fit(
    X_train_scaled,
    y_train,
    epochs=150,                 # Número máximo de épocas
    batch_size=32,
    validation_split=0.2,
    callbacks=[early_stopper],
    class_weight=class_weight_dict, # <-- APLICANDO O BALANCEAMENTO
    verbose=2                   # Mostra o log de treino
)

print("--- Treinamento Concluído ---")


# ==============================================================================
# --- 7. Avaliando o Modelo 7 (Rede Neural) ---
# ==============================================================================
print("\n--- 7. Avaliando o Modelo 7 (Rede Neural) no conjunto de TESTE ---")

# Avaliar a perda e métricas finais nos dados de TESTE (escalados)
test_loss, test_accuracy, test_recall = model.evaluate(X_test_scaled, y_test, verbose=0)
print(f"Acurácia (Modelo 7): {test_accuracy:.4f}")
print(f"Recall (Modelo 7):   {test_recall:.4f}") # Recall na classe positiva

# Obter as previsões
# As previsões são probabilidades (ex: 0.15, 0.88, 0.45)
y_pred_proba = model.predict(X_test_scaled)

# Converter probabilidades em classes (0 ou 1)
# Usamos o limiar (threshold) padrão de 0.5
y_pred_nn = (y_pred_proba > 0.5).astype("int32")

# Exibir Relatório de Classificação
print("\nRelatório de Classificação (Modelo 7 - Rede Neural):")
print(classification_report(y_test, y_pred_nn, target_names=['negativo (0)', 'positivo (1)']))

# Exibir Matriz de Confusão
print("\nMatriz de Confusão (Modelo 7 - Rede Neural):")
cm_nn = confusion_matrix(y_test, y_pred_nn)
print(f"            [Prev. Neg] [Prev. Pos]")
print(f"[Real Neg]  {cm_nn[0][0]:>10} {cm_nn[0][1]:>10}")
print(f"[Real Pos]  {cm_nn[1][0]:>10} {cm_nn[1][1]:>10}")

2025-10-28 07:47:46.982556: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2025-10-28 07:47:46.998629: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-10-28 07:47:47.492025: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-10-28 07:47:51.672673: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off,

--- 1. Carregando e Pré-processando os Dados ---
--- Pré-processamento Concluído. 43 features criadas. ---

--- 2. Dividindo os Dados (Treino e Teste) ---
Tamanho do conjunto de Treino: 319
Tamanho do conjunto de Teste:  137

--- 3. Aplicando StandardScaler (Fundamental para NNs) ---
--- Dados escalados com sucesso ---

--- 4. Calculando Pesos da Classe (class_weight) ---
Total de Treino: 319, Negativos: 224, Positivos: 95
Peso para classe 0 (negativo): 0.7121
Peso para classe 1 (positivo): 1.6789

--- 5. Modelo 7: Construindo a Arquitetura da Rede Neural ---


/home/pontes/.cache/pypoetry/virtualenvs/ia-model-OlOOztcw-py3.12/lib/python3.12/site-packages/keras/src/layers/core/dense.py:95: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
2025-10-28 07:47:54.201412: E external/local_xla/xla/stream_executor/cuda/cuda_platform.cc:51] failed call to cuInit: INTERNAL: CUDA error: Failed call to cuInit: UNKNOWN ERROR (303)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense (Dense)                   │ (None, 32)             │         1,408 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 32)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 16)             │           528 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 16)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 1)              │            17 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 1,953 (7.63 KB)

 Trainable params: 1,953 (7.63 KB)

 Non-trainable params: 0 (0.00 B)


--- 6. Treinando o Modelo 7 (Rede Neural) ---
Epoch 1/150
8/8 - 1s - 130ms/step - accuracy: 0.5412 - loss: 0.8620 - recall: 0.3289 - val_accuracy: 0.5469 - val_loss: 0.6703 - val_recall: 0.3158
Epoch 2/150
8/8 - 0s - 9ms/step - accuracy: 0.5098 - loss: 0.8984 - recall: 0.4211 - val_accuracy: 0.5312 - val_loss: 0.6712 - val_recall: 0.2632
Epoch 3/150
8/8 - 0s - 9ms/step - accuracy: 0.5804 - loss: 0.8342 - recall: 0.4342 - val_accuracy: 0.5156 - val_loss: 0.6714 - val_recall: 0.2632
Epoch 4/150
8/8 - 0s - 9ms/step - accuracy: 0.5725 - loss: 0.8097 - recall: 0.4605 - val_accuracy: 0.5000 - val_loss: 0.6748 - val_recall: 0.2632
Epoch 5/150
8/8 - 0s - 9ms/step - accuracy: 0.5412 - loss: 0.7803 - recall: 0.5132 - val_accuracy: 0.5312 - val_loss: 0.6776 - val_recall: 0.3684
Epoch 6/150
8/8 - 0s - 8ms/step - accuracy: 0.5765 - loss: 0.8444 - recall: 0.4605 - val_accuracy: 0.5156 - val_loss: 0.6798 - val_recall: 0.3684
Epoch 7/150
8/8 - 0s - 9ms/step - accuracy: 0.5412 - loss: 0.8292 - recall: